In [ ]:
!pip install -U langgraph langchain langchain_openai langchain_experimental langsmith pandas

In [ ]:
from typing import TypedDict, Annotated, List, Union, Tuple
from langchain_core.agents import AgentAction, AgentFinish
import operator


class JarvisState(TypedDict):
    input: str
    agent_out: Union[AgentAction, AgentFinish, None]
    protocol: dict
    intermediate_steps: Annotated[List[Tuple[AgentAction, str]], operator.add]

In [135]:
from langchain_core.tools import tool
from pydantic import BaseModel
from langchain_openai import ChatOpenAI

from enum import Enum, auto

class CoffeeQuery(BaseModel):
    type: str
    serving_size: str


@tool("get_coffee")
def make_coffee_tool(query: CoffeeQuery):
    """Prepares coffee for of the given type and serving_size.
    `type` can be any one of (AMERICANO, CAPPUCCINO, MOCHA)
    `serving_size` can be any of (REGULAR, LARGE, GRANDE)
    Cannot be used to for any other topics."""

    return f'Here is your {query.type} in you favorite {query.serving_size} cup.'

@tool("get_chat_response")
def role_play_chat_tool(query: str):
    """Generates a chat response on any given topic. query must be provided
    in natural language and be verbose.
    """
    llm = ChatOpenAI(temperature=0)
    prompt = f"You are Ultron. You are an expert in all topics. You respond to all queies in third-person way.\n\n\nquery: {query}"
    response = llm.invoke(prompt)
    return response.content


@tool("send_email")
def send_email_tool(
    recipient: str,
    subject: str,
    body: str
):
    """Sends email to the given recipient with a valid subject and email body"""

    print('sending email.....', recipient, subject, body)
    return f"Email:\n\n\n To: {recipient}\n\nSubject: {subject}\n\nBody: {body}"


@tool("special_protocols")
def special_protocols_tool(
    protocol_name: str
):
    """Checks and Runs the given protocol for the selected target. Must be used only while triggering a protocol
    """

    # protocol_name can be one of (theta, house-party, training-wheels)
    if protocol_name.lower() not in ['theta', 'house-party', 'training-wheels']:
        return { 'message': f'Cannot find the {protocol_name} protocol, Tony!', 'registered': False, 'protocol_name': protocol_name }

    return { 'message': f'Okay, triggering {protocol_name}', 'registered': True, 'protocol_name': protocol_name }


In [ ]:
import os
from langchain.agents import create_openai_tools_agent
from langchain import hub
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

prompt = hub.pull("hwchase17/openai-functions-agent")
user_name = 'Tony Stark'
prompt.messages[0].prompt.template = f'You are a helpful assistant named Jarvis, you answer to {user_name}'

query_agent_runnable = create_openai_tools_agent(
    llm=llm,
    tools=[make_coffee_tool, role_play_chat_tool, send_email_tool, special_protocols_tool],
    prompt=prompt
)


In [ ]:
inputs = {
    "input": "Hey jarvis, send email to im.hulk@avengers.com saying wassup Bruce, hope alls well, want to discuss about Vibranium, meet me",
    "intermediate_steps": []
}
agent_out = query_agent_runnable.invoke(inputs)
agent_out

inputs = {
    "input": "Jarvis, tell me about who is Zeus?",
    "intermediate_steps": []
}
agent_out = query_agent_runnable.invoke(inputs)
agent_out

inputs = {
    "input": "where is Delhi?",
    "intermediate_steps": []
}
agent_out = query_agent_runnable.invoke(inputs)
agent_out

In [ ]:
import json

def capture_task(state: JarvisState):
    print("capture_task")
    agent_out = query_agent_runnable.invoke(state)
    
    return {"agent_out": agent_out}

def check_task_type_router(state: JarvisState):
    print("check_task_type_router")
    action = state["agent_out"]
    
    if "tool_calls" in action[-1].message_log[-1].additional_kwargs:
        tool_call = action[-1].message_log[-1].additional_kwargs["tool_calls"][-1]
        function_name = tool_call["function"]["name"] 

        if function_name == "special_protocols":
            return "is_special_protocols"
    
    return "is_routine_task"


def execute_task(state: JarvisState):
    print("execute_task")
    action = state["agent_out"]
    tool_call = action[-1].message_log[-1].additional_kwargs["tool_calls"][-1]
    function_name = tool_call["function"]["name"] 
    function_args = json.loads(tool_call["function"]["arguments"])

    if function_name == "get_coffee":
        return { "agent_out": make_coffee_tool(function_args)}

    if function_name == "get_chat_response":
        return { "agent_out": role_play_chat_tool(function_args) }

    if function_name == "send_email":
        return { "agent_out": send_email_tool(function_args) }
    

# answer_formatter_llm = formatter_llm.bind_tools([response_formatter_tool])

def check_for_protocol(state: JarvisState):
    print('check_for_protocol')

    action = state["agent_out"]
    tool_call = action[-1].message_log[-1].additional_kwargs["tool_calls"][-1]
    function_args = json.loads(tool_call["function"]["arguments"])
    
    protocol_registered = special_protocols_tool(function_args)

    return { 'agent_out': protocol_registered['message'], 'protocol': protocol_registered }


def protocol_router(state: JarvisState):
    print('protocol_router')
    protocol_data = state['protocol']

    if protocol_data['registered']:
        return 'registered'

    return 'not_registered'

import random
def reject_protocol(state: JarvisState):
    print('reject_protocol')
    protocol_data = state['protocol']

    print('protocol_data', protocol_data, protocol_data['protocol_name'])

    return { 'agent_out': f'Hey Sorry Tony, you need to register the protocol {protocol_data['protocol_name']} first.' }

import random
def execute_protocol(state: JarvisState):
    print('execute_protocol')
    protocol_data = state['protocol']
    random_number = random.randint(1, 10)
    if random_number%2 == 0:
        return { agent_out: f'Failed to execute protocol {protocol_data['protocol_name']}' }
    
    return { 'agent_out': f'Executing protocol {protocol_data['protocol_name']}' }

In [ ]:
from langgraph.graph import StateGraph, END, START

graph = StateGraph(JarvisState)

graph.add_node("capture_task", capture_task)
graph.add_node("execute_task", execute_task)
graph.add_node("check_for_protocol", check_for_protocol)
graph.add_node("execute_protocol", execute_protocol)
graph.add_node("reject_protocol", reject_protocol)


graph.set_entry_point("capture_task")

# conditional edges are controlled by our router
graph.add_conditional_edges(
    source="capture_task",  # where in graph to start
    path=check_task_type_router,  # function to determine which node is called
    path_map={
        'is_special_protocols': "check_for_protocol",
        'is_routine_task': "execute_task"
    }
)
graph.add_edge("execute_task", END)

graph.add_conditional_edges(
    source="check_for_protocol",  # where in graph to start
    path=protocol_router,  # function to determine which node is called
    path_map={
        'registered': 'execute_protocol',
        'not_registered': 'reject_protocol'
    }
)

graph.add_edge("execute_protocol", END)
graph.add_edge("reject_protocol", END)



runnable = graph.compile()


In [ ]:
from IPython.display import Image, display

try:
    display(Image(runnable.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:

out = runnable.invoke({
    "input": "jarvis please make me a regular cappuchina?",
    "chat_history": []
})
print(out["agent_out"])


In [ ]:

out = runnable.invoke({
    "input": "Tell me about Anubis",
    "chat_history": []
})
print(out["agent_out"])
